<a href="https://colab.research.google.com/github/swt02026/CallingGoInJava/blob/master/Java_Call_go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

In [0]:
!apt install golang-go libzmq3-dev file
%env GOPATH=/root/go
!go get -u github.com/gopherdata/gophernotes
!cp ~/go/bin/gophernotes /usr/bin/
!mkdir /usr/local/share/jupyter/kernels/gophernotes
!cp ~/go/src/github.com/gopherdata/gophernotes/kernel/* \
       /usr/local/share/jupyter/kernels/gophernotes

In [0]:
%cd /content

In [0]:
!git clone https://github.com/swt02026/CallingGoInJava.git go_java

In [0]:
%cd go_java

In [0]:
%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:.

In [0]:
!wget https://github.com/oracle/graal/releases/download/vm-1.0.0-rc15/graalvm-ce-1.0.0-rc15-linux-amd64.tar.gz

In [0]:
!tar -xvzf graalvm-ce-1.0.0-rc15-linux-amd64.tar.gz

In [8]:
%env GRAALVM=graalvm-ce-1.0.0-rc15

env: GRAALVM=graalvm-ce-1.0.0-rc15


In [0]:
!${GRAALVM}/bin/java -version

# JNI with go build

In [10]:
!cat HelloWorld.java;

class HelloWorld {
    private native void heavyWork(int num);

    public static void main(String[] args) {
        final int num = Integer.valueOf(args[0]);
        new HelloWorld().heavyWork(num);
    }

    static {
        System.loadLibrary("HelloWorld");
    }
}


In [11]:
!find / -name jni.h

find: ‘/proc/1272’: No such file or directory
find: ‘/proc/1273’: No such file or directory
/usr/include/x86_64-linux-gnu/libavcodec/jni.h
/usr/lib/jvm/java-8-openjdk-amd64/include/jni.h
/content/go_java/graalvm-ce-1.0.0-rc15/include/jni.h


In [12]:
!${GRAALVM}/bin/javah -jni HelloWorld; ${GRAALVM}/bin/javac HelloWorld.java; ls -alrt;

total 335068
-rw-r--r-- 1 root root 343000283 Apr  5 11:57 graalvm-ce-1.0.0-rc15-linux-amd64.tar.gz
drwxr-xr-x 1 root root      4096 Apr 23 11:34 ..
-rw-r--r-- 1 root root        17 Apr 23 11:34 README.md
-rw-r--r-- 1 root root     45270 Apr 23 11:34 Java_Call_go.ipynb
-rw-r--r-- 1 root root       270 Apr 23 11:34 HelloWorld.java
-rw-r--r-- 1 root root       652 Apr 23 11:34 HelloWorldGraalVM.java
-rw-r--r-- 1 root root       166 Apr 23 11:34 HelloWorld.go
-rw-r--r-- 1 root root       259 Apr 23 11:34 HelloWorldCallGO.c
-rw-r--r-- 1 root root       240 Apr 23 11:34 HelloWorld.c
-rw-r--r-- 1 root root       357 Apr 23 11:34 Headers.java
drwxr-xr-x 8 root root      4096 Apr 23 11:34 .git
drwxr-xr-x 8 root root      4096 Apr 23 11:34 graalvm-ce-1.0.0-rc15
-rw-r--r-- 1 root root       392 Apr 23 11:35 HelloWorld.h
drwxr-xr-x 4 root root      4096 Apr 23 11:35 .
-rw-r--r-- 1 root root       580 Apr 23 11:35 HelloWorld.class


In [13]:
!cat HelloWorld.h

/* DO NOT EDIT THIS FILE - it is machine generated */
#include <jni.h>
/* Header for class HelloWorld */

#ifndef _Included_HelloWorld
#define _Included_HelloWorld
#ifdef __cplusplus
extern "C" {
#endif
/*
 * Class:     HelloWorld
 * Method:    heavyWork
 * Signature: (I)V
 */
JNIEXPORT void JNICALL Java_HelloWorld_heavyWork
  (JNIEnv *, jobject, jint);

#ifdef __cplusplus
}
#endif
#endif


In [14]:
! cat  HelloWorld.c

#include <jni.h>

#include "HelloWorld.h"
#include <stdio.h>

JNIEXPORT void JNICALL Java_HelloWorld_heavyWork(JNIEnv *env, jobject obj,
                                                 jint num) {

  printf("heavy work in c %d\n", num);
}


In [15]:
!gcc -I ${GRAALVM}/include/ -I ${GRAALVM}/include/linux -fpie -shared -o libHelloWorld.so HelloWorld.c;ls -alrt

total 335076
-rw-r--r-- 1 root root 343000283 Apr  5 11:57 graalvm-ce-1.0.0-rc15-linux-amd64.tar.gz
drwxr-xr-x 1 root root      4096 Apr 23 11:34 ..
-rw-r--r-- 1 root root        17 Apr 23 11:34 README.md
-rw-r--r-- 1 root root     45270 Apr 23 11:34 Java_Call_go.ipynb
-rw-r--r-- 1 root root       270 Apr 23 11:34 HelloWorld.java
-rw-r--r-- 1 root root       652 Apr 23 11:34 HelloWorldGraalVM.java
-rw-r--r-- 1 root root       166 Apr 23 11:34 HelloWorld.go
-rw-r--r-- 1 root root       259 Apr 23 11:34 HelloWorldCallGO.c
-rw-r--r-- 1 root root       240 Apr 23 11:34 HelloWorld.c
-rw-r--r-- 1 root root       357 Apr 23 11:34 Headers.java
drwxr-xr-x 8 root root      4096 Apr 23 11:34 .git
drwxr-xr-x 8 root root      4096 Apr 23 11:34 graalvm-ce-1.0.0-rc15
-rw-r--r-- 1 root root       392 Apr 23 11:35 HelloWorld.h
-rw-r--r-- 1 root root       580 Apr 23 11:35 HelloWorld.class
drwxr-xr-x 4 root root      4096 Apr 23 11:35 .
-rwxr-xr-x 1 root root      7928 Apr 23 11:35 libHelloWorld.so


In [16]:
!java HelloWorld 13245

heavy work in c 13245


In [17]:
!cat HelloWorld.go

package main

import "C"  
import "fmt"

//export HeavyWorkFromGo
func HeavyWorkFromGo(num int) {
    fmt.Printf("HeavyWork in Go: %d\n", num)
}

func main(){
    
}


In [18]:
!go build -buildmode=c-shared -o libHelloWorldCallGO.so HelloWorld.go;ls -alrt

total 337568
-rw-r--r-- 1 root root 343000283 Apr  5 11:57 graalvm-ce-1.0.0-rc15-linux-amd64.tar.gz
drwxr-xr-x 1 root root      4096 Apr 23 11:34 ..
-rw-r--r-- 1 root root        17 Apr 23 11:34 README.md
-rw-r--r-- 1 root root     45270 Apr 23 11:34 Java_Call_go.ipynb
-rw-r--r-- 1 root root       270 Apr 23 11:34 HelloWorld.java
-rw-r--r-- 1 root root       652 Apr 23 11:34 HelloWorldGraalVM.java
-rw-r--r-- 1 root root       166 Apr 23 11:34 HelloWorld.go
-rw-r--r-- 1 root root       259 Apr 23 11:34 HelloWorldCallGO.c
-rw-r--r-- 1 root root       240 Apr 23 11:34 HelloWorld.c
-rw-r--r-- 1 root root       357 Apr 23 11:34 Headers.java
drwxr-xr-x 8 root root      4096 Apr 23 11:34 .git
drwxr-xr-x 8 root root      4096 Apr 23 11:34 graalvm-ce-1.0.0-rc15
-rw-r--r-- 1 root root       392 Apr 23 11:35 HelloWorld.h
-rw-r--r-- 1 root root       580 Apr 23 11:35 HelloWorld.class
-rwxr-xr-x 1 root root      7928 Apr 23 11:35 libHelloWorld.so
-rw-r--r-- 1 root root      1521 Apr 23 11:35 libHel

In [19]:
!cat HelloWorldCallGO.c

#include <jni.h>

#include "HelloWorld.h"
#include <stdio.h>
#include "libHelloWorldCallGO.h"

JNIEXPORT void JNICALL Java_HelloWorld_heavyWork(JNIEnv *env, jobject obj,
                                                 jint num)
{
    HeavyWorkFromGo(num);
}


In [20]:
!gcc -o libHelloWorld.so  -I ${GRAALVM}/include/ -I ${GRAALVM}/include/linux  -fpie -shared HelloWorldCallGO.c -L. -lHelloWorldCallGO; ls -alrt

total 337568
-rw-r--r-- 1 root root 343000283 Apr  5 11:57 graalvm-ce-1.0.0-rc15-linux-amd64.tar.gz
drwxr-xr-x 1 root root      4096 Apr 23 11:34 ..
-rw-r--r-- 1 root root        17 Apr 23 11:34 README.md
-rw-r--r-- 1 root root     45270 Apr 23 11:34 Java_Call_go.ipynb
-rw-r--r-- 1 root root       270 Apr 23 11:34 HelloWorld.java
-rw-r--r-- 1 root root       652 Apr 23 11:34 HelloWorldGraalVM.java
-rw-r--r-- 1 root root       166 Apr 23 11:34 HelloWorld.go
-rw-r--r-- 1 root root       259 Apr 23 11:34 HelloWorldCallGO.c
-rw-r--r-- 1 root root       240 Apr 23 11:34 HelloWorld.c
-rw-r--r-- 1 root root       357 Apr 23 11:34 Headers.java
drwxr-xr-x 8 root root      4096 Apr 23 11:34 .git
drwxr-xr-x 8 root root      4096 Apr 23 11:34 graalvm-ce-1.0.0-rc15
-rw-r--r-- 1 root root       392 Apr 23 11:35 HelloWorld.h
-rw-r--r-- 1 root root       580 Apr 23 11:35 HelloWorld.class
-rw-r--r-- 1 root root      1521 Apr 23 11:35 libHelloWorldCallGO.h
-rw-r--r-- 1 root root   2544704 Apr 23 11:35 l

In [21]:
!java HelloWorld 1324555555

HeavyWork in Go: 1324555555


In [22]:
!file HelloWorld.class

HelloWorld.class: compiled Java class data, version 52.0 (Java 1.8)


# GraalVM

https://cornerwings.github.io/2018/07/graal-native-methods/


JNI差異、JNI(env、OBJ)、Call Convension
Native Image 
poltgon
go buildmode
output binary差異
sulong llvm bitcode

In [23]:
!cat Headers.java

import java.io.*;
import org.graalvm.polyglot.*;
import org.graalvm.nativeimage.*;
import java.util.*;

import org.graalvm.nativeimage.c.CContext;

public class Headers implements CContext.Directives {

    @Override
    public List<String> getHeaderFiles() {
        return Collections.singletonList("\"/content/go_java/libHelloWorldCallGO.h\"");
    }

}


In [24]:
!cat HelloWorldGraalVM.java

import java.io.*;
import org.graalvm.polyglot.*;
import org.graalvm.nativeimage.*;
import java.util.*;

import org.graalvm.nativeimage.c.CContext;
import org.graalvm.nativeimage.c.function.CLibrary;
import org.graalvm.nativeimage.c.function.CFunction;
import org.graalvm.nativeimage.c.function.CFunction.Transition;

@CContext(Headers.class)
@CLibrary("HelloWorldCallGO")
public class HelloWorldGraalVM {

    @CFunction(transition = Transition.NO_TRANSITION)
    private static native void HeavyWorkFromGo(int num);

    public static void main(String[] args) {
        final int num = Integer.valueOf(args[0]);
        HeavyWorkFromGo(num);

    }
}


In [25]:
!${GRAALVM}/bin/javac Headers.java HelloWorldGraalVM.java; ls -alrt

total 337576
-rw-r--r-- 1 root root 343000283 Apr  5 11:57 graalvm-ce-1.0.0-rc15-linux-amd64.tar.gz
drwxr-xr-x 1 root root      4096 Apr 23 11:34 ..
-rw-r--r-- 1 root root        17 Apr 23 11:34 README.md
-rw-r--r-- 1 root root     45270 Apr 23 11:34 Java_Call_go.ipynb
-rw-r--r-- 1 root root       270 Apr 23 11:34 HelloWorld.java
-rw-r--r-- 1 root root       652 Apr 23 11:34 HelloWorldGraalVM.java
-rw-r--r-- 1 root root       166 Apr 23 11:34 HelloWorld.go
-rw-r--r-- 1 root root       259 Apr 23 11:34 HelloWorldCallGO.c
-rw-r--r-- 1 root root       240 Apr 23 11:34 HelloWorld.c
-rw-r--r-- 1 root root       357 Apr 23 11:34 Headers.java
drwxr-xr-x 8 root root      4096 Apr 23 11:34 .git
drwxr-xr-x 8 root root      4096 Apr 23 11:34 graalvm-ce-1.0.0-rc15
-rw-r--r-- 1 root root       392 Apr 23 11:35 HelloWorld.h
-rw-r--r-- 1 root root       580 Apr 23 11:35 HelloWorld.class
-rw-r--r-- 1 root root      1521 Apr 23 11:35 libHelloWorldCallGO.h
-rw-r--r-- 1 root root   2544704 Apr 23 11:35 l

In [26]:
!${GRAALVM}/bin/native-image -H:+ReportExceptionStackTraces -cp . -Djava.library.path=. -H:CLibraryPath=. HelloWorldGraalVM; ls -alrt

Build on Server(pid: 1522, port: 40105)*
[helloworldgraalvm:1522]    classlist:   3,335.32 ms
[helloworldgraalvm:1522]        (cap):   2,067.10 ms
[helloworldgraalvm:1522]        setup:   4,298.68 ms
[helloworldgraalvm:1522]   (typeflow):   5,716.32 ms
[helloworldgraalvm:1522]    (objects):   1,564.16 ms
[helloworldgraalvm:1522]   (features):     310.37 ms
[helloworldgraalvm:1522]     analysis:   7,732.69 ms
[helloworldgraalvm:1522]     universe:     389.00 ms
[helloworldgraalvm:1522]      (parse):   1,036.08 ms
[helloworldgraalvm:1522]     (inline):   2,554.07 ms
[helloworldgraalvm:1522]    (compile):  13,300.11 ms
[helloworldgraalvm:1522]      compile:  17,384.83 ms
[helloworldgraalvm:1522]        image:     681.37 ms
[helloworldgraalvm:1522]        write:     173.39 ms
[helloworldgraalvm:1522]      [total]:  34,215.05 ms
total 340040
-rw-r--r-- 1 root root 343000283 Apr  5 11:57 graalvm-ce-1.0.0-rc15-linux-amd64.tar.gz
drwxr-xr-x 1 root root      4096 Apr 23 11:34 ..
-rw-r--r-- 1 ro

In [27]:
! ./helloworldgraalvm 123445555

HeavyWork in Go: 123445555


In [28]:
!file helloworldgraalvm

helloworldgraalvm: ELF 64-bit LSB shared object, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/l, for GNU/Linux 3.2.0, BuildID[sha1]=9cbf15ed688539fc39466544a497d245e3477897, with debug_info, not stripped


In [29]:
!ls -lh helloworldgraalvm

-rwxr-xr-x 1 root root 2.5M Apr 23 11:36 helloworldgraalvm
